In [1]:
# Basic modules
import cells
import numpy as np
import matplotlib.pyplot as plt
#from numpy.matlib import repmat
from numpy.random import randn

# Load initial conditions

In [2]:
vectors = np.genfromtxt('initial_conditions.csv', delimiter=',')
R = vectors[:,0:3] # Cells radiovectors
P = vectors[:,3:6] # ABP vectors
Q = vectors[:,6:9] # PCP vectors

In [ ]:
# Matplotlib -------------------------------------
plt.close()
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection="3d")
# Plot population of cells
ax.scatter(R[:,0], R[:,1], R[:,2], 
           marker="o", s=200, color="#69409e", alpha=1, label="Population")
ax.set_xlabel("X", weight="bold")
ax.set_ylabel("Y", weight="bold")
ax.set_zlabel("Z", weight="bold")
ax.legend()
fig.canvas.draw()

# Simulation

## Strength of attractive/repulsive potential

Calculate the weights of the attraction coefficients according to the formulae:

\begin{equation}
\begin{split}
S_1 &= (p^i \wedge r^{ij}) \cdot (p^j \wedge r^{ij}) = p^{i}_{\mu} p^{j}_{\mu} r^{ij}_{\mu}r^{ij}_{\mu} - p^{i}_{\mu}r^{ij}_{\mu} p^{j}_{\nu}r^{ij}_{\nu} \\
S_2 &= (p^i \wedge q^{i}) \cdot (p^j \wedge q^{j}) = p^{i}_{\mu} p^{j}_{\mu} q^{i}_{\nu}q^{j}_{\nu} - p^{i}_{\mu}q^{j}_{\mu} p^{j}_{\nu}q^{i}_{\nu} \\
S_3 &= (q^i \wedge r^{ij}) \cdot (q^j \wedge r^{ij}) = q^{i}_{\mu} q^{j}_{\mu} r^{ij}_{\mu}r^{ij}_{\mu} - q^{i}_{\mu}r^{ij}_{\mu} q^{j}_{\nu}r^{ij}_{\nu} \\
\end{split}
\end{equation}

The strength of attraction is $S$, where
\begin{equation}
S = \sigma_1 S_1 + \sigma_2 S_2 + \sigma_3 S_3, \qquad \sigma_1 + \sigma_2 + \sigma_3 = 1.
\end{equation}

## Simulation on Vispy

In [3]:
%gui qt
import vispy
from vispy.scene import visuals
from vispy import app

In [5]:
# Define canvas and enable interactive PoV
canvas = vispy.scene.SceneCanvas(keys='interactive', show=True, bgcolor='black')
view = canvas.central_widget.add_view()
view.camera = 'turntable'

r = R
p = P
q = Q
def simulation(t):
    global r, p, q
    # Time evolution parameters
    dt = 0.1
    # Integration over time domain
    print("Simulation in progress... Evolution step = ", t)
    dr, dp, dq = cells.evolution(r, p, q)
    r = r + dt*dr + np.random.normal(0, 1e-5, size=(len(r),3))
    p = p + dt*dp + np.random.normal(0, 1e-5, size=(len(p),3))
    q = q + dt*dq + np.random.normal(0, 1e-5, size=(len(q),3))
    p = cells.unitary(p)
    q = cells.unitary(q)
    return r + p

scatter = visuals.Markers(scaling=True, spherical=True)
view.add(scatter)
axis = visuals.XYZAxis(parent=view.scene)

t = 0
def update(event):
    global t, scatter
    scatter.set_data(simulation(t), face_color="#B34E66", size=200,
                     edge_color='white', edge_width=0)
    t += 1

time = app.Timer()
time.connect(update)
time.start(interval=0.05, iterations=1000)
time.events.stop.connect(lambda x: app.quit())

if __name__ == '__main__':
    canvas.show()
    app.run()

Simulation in progress... Evolution step =  0
Simulation in progress... Evolution step =  1
Simulation in progress... Evolution step =  2
Simulation in progress... Evolution step =  3
Simulation in progress... Evolution step =  4
Simulation in progress... Evolution step =  5
Simulation in progress... Evolution step =  6
Simulation in progress... Evolution step =  7
Simulation in progress... Evolution step =  8
Simulation in progress... Evolution step =  9
Simulation in progress... Evolution step =  10
Simulation in progress... Evolution step =  11
Simulation in progress... Evolution step =  12
Simulation in progress... Evolution step =  13
Simulation in progress... Evolution step =  14
Simulation in progress... Evolution step =  15
Simulation in progress... Evolution step =  16
Simulation in progress... Evolution step =  17
Simulation in progress... Evolution step =  18
Simulation in progress... Evolution step =  19
Simulation in progress... Evolution step =  20
Simulation in progress.

Simulation in progress... Evolution step =  173
Simulation in progress... Evolution step =  174
Simulation in progress... Evolution step =  175
Simulation in progress... Evolution step =  176
Simulation in progress... Evolution step =  177
Simulation in progress... Evolution step =  178
Simulation in progress... Evolution step =  179
Simulation in progress... Evolution step =  180
Simulation in progress... Evolution step =  181
Simulation in progress... Evolution step =  182
Simulation in progress... Evolution step =  183
Simulation in progress... Evolution step =  184
Simulation in progress... Evolution step =  185
Simulation in progress... Evolution step =  186
Simulation in progress... Evolution step =  187
Simulation in progress... Evolution step =  188
Simulation in progress... Evolution step =  189
Simulation in progress... Evolution step =  190
Simulation in progress... Evolution step =  191
Simulation in progress... Evolution step =  192
Simulation in progress... Evolution step

Simulation in progress... Evolution step =  345
Simulation in progress... Evolution step =  346
Simulation in progress... Evolution step =  347
Simulation in progress... Evolution step =  348
Simulation in progress... Evolution step =  349
Simulation in progress... Evolution step =  350
Simulation in progress... Evolution step =  351
Simulation in progress... Evolution step =  352
Simulation in progress... Evolution step =  353
Simulation in progress... Evolution step =  354
Simulation in progress... Evolution step =  355
Simulation in progress... Evolution step =  356
Simulation in progress... Evolution step =  357
Simulation in progress... Evolution step =  358
Simulation in progress... Evolution step =  359
Simulation in progress... Evolution step =  360
Simulation in progress... Evolution step =  361
Simulation in progress... Evolution step =  362
Simulation in progress... Evolution step =  363
Simulation in progress... Evolution step =  364
Simulation in progress... Evolution step

Simulation in progress... Evolution step =  518
Simulation in progress... Evolution step =  519
Simulation in progress... Evolution step =  520
Simulation in progress... Evolution step =  521
Simulation in progress... Evolution step =  522
Simulation in progress... Evolution step =  523
Simulation in progress... Evolution step =  524
Simulation in progress... Evolution step =  525
Simulation in progress... Evolution step =  526
Simulation in progress... Evolution step =  527
Simulation in progress... Evolution step =  528
Simulation in progress... Evolution step =  529
Simulation in progress... Evolution step =  530
Simulation in progress... Evolution step =  531
Simulation in progress... Evolution step =  532
Simulation in progress... Evolution step =  533
Simulation in progress... Evolution step =  534
Simulation in progress... Evolution step =  535
Simulation in progress... Evolution step =  536
Simulation in progress... Evolution step =  537
Simulation in progress... Evolution step

Simulation in progress... Evolution step =  692
Simulation in progress... Evolution step =  693
Simulation in progress... Evolution step =  694
Simulation in progress... Evolution step =  695
Simulation in progress... Evolution step =  696
Simulation in progress... Evolution step =  697
Simulation in progress... Evolution step =  698
Simulation in progress... Evolution step =  699
Simulation in progress... Evolution step =  700
Simulation in progress... Evolution step =  701
Simulation in progress... Evolution step =  702
Simulation in progress... Evolution step =  703
Simulation in progress... Evolution step =  704
Simulation in progress... Evolution step =  705
Simulation in progress... Evolution step =  706
Simulation in progress... Evolution step =  707
Simulation in progress... Evolution step =  708
Simulation in progress... Evolution step =  709
Simulation in progress... Evolution step =  710
Simulation in progress... Evolution step =  711
Simulation in progress... Evolution step

Simulation in progress... Evolution step =  864
Simulation in progress... Evolution step =  865
Simulation in progress... Evolution step =  866
Simulation in progress... Evolution step =  867
Simulation in progress... Evolution step =  868
Simulation in progress... Evolution step =  869
Simulation in progress... Evolution step =  870
Simulation in progress... Evolution step =  871
Simulation in progress... Evolution step =  872
Simulation in progress... Evolution step =  873
Simulation in progress... Evolution step =  874
Simulation in progress... Evolution step =  875
Simulation in progress... Evolution step =  876
Simulation in progress... Evolution step =  877
Simulation in progress... Evolution step =  878
Simulation in progress... Evolution step =  879
Simulation in progress... Evolution step =  880
Simulation in progress... Evolution step =  881
Simulation in progress... Evolution step =  882
Simulation in progress... Evolution step =  883
Simulation in progress... Evolution step

## Integration over time + output files

In [ ]:
# Avoid overwriting vectors with initial datum
r = R
p = P
q = Q
# Time evolution parameters
t = 0
dt = 0.1
# Integration over time domain
while t < 1000:
    print("Recording data... Evolution step = ", t)
    dr, dp, dq = cells.evolution(r, p, q)
    r = r + dt*dr + np.random.normal(0, 1e-5, size=(len(r),3))
    p = p + dt*dp + np.random.normal(0, 1e-5, size=(len(p),3))
    q = q + dt*dq + np.random.normal(0, 1e-5, size=(len(q),3))
    p = cells.unitary(p)
    q = cells.unitary(q)
    np.savetxt('output/r' + str(t) + '.csv', r, delimiter=',')
    np.savetxt('output/p' + str(t) + '.csv', p, delimiter=',')
    np.savetxt('output/q' + str(t) + '.csv', q, delimiter=',')
    t = t + 1

# *Matplotlib animation (not efficient, only for debugging)

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection="3d")

def animate(i):
    r_t = np.genfromtxt('output/r' + str(i) + '.csv', delimiter=',')
    p_t = np.genfromtxt('output/p' + str(i) + '.csv', delimiter=',')
    q_t = np.genfromtxt('output/q' + str(i) + '.csv', delimiter=',')

    ax.clear()
    ax.scatter(r_t[:,0] + p_t[:,0], r_t[:,1] + p_t[:,1], r_t[:,2] + p_t[:,2],
               marker="o", s=200, color="#69409e")
    ax.set_xlim([-8,8])
    ax.set_ylim([-8,8])
    ax.set_zlim([-8,8])

ani = animation.FuncAnimation(fig, animate, interval=5)
plt.show()